In [1]:
# There are two small differences to be made in RNN_sentiment_analysis.py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.distributed as dist
import torch.backends.cudnn as cudnn

import time
import os
import sys
import io

from RNN_model import RNN_model

################## Args ################
if 1: # len(sys.argv) == (1 + 7):
    opt = 'adam' # str(sys.argv[1]) # opt = 'sgd' or 'adam'
    LR = 0.001 # float(sys.argv[2])
    batch_size = 200 # int(sys.argv[3])
    vocab_size = 8000 # int(sys.argv[4])
    no_of_hidden_units = 500 # int(sys.argv[5])
    no_of_epochs = 21 # int(sys.argv[6])
    sequence_length = 50 # int(sys.argv[7])
    print('optimizer: %s | LR: %f | batch_size: %d | vocab_size: %d | no_of_hidden_units: %d | no_of_epochs: %d | sequence_length_train: %d' % 
        (opt, LR, batch_size, vocab_size, no_of_hidden_units, no_of_epochs, sequence_length))
else:
    raise Exception('Incorrect number of arguments. Program exit!')

########################################

print('RNN 3C')
is_cuda = torch.cuda.is_available()
print('Cuda is', is_cuda)


imdb_dictionary = np.load('../preprocessed_data/imdb_dictionary.npy')
# vocab_size = 8000

# The last three lines of code just grab the first 25,000 sequences
#  and creates a vector for the labels (the first 125000 are labeled 1 
# for positive and the last 12500 are labeled 0 for negative).
x_train = []
with io.open('../preprocessed_data/imdb_train.txt','r',encoding='utf-8') as f:
    lines = f.readlines()
for line in lines:
    line = line.strip()
    line = line.split(' ')
    line = np.asarray(line,dtype=np.int)

    line[line>vocab_size] = 0

    x_train.append(line)
x_train = x_train[0:25000]
y_train = np.zeros((25000,))
y_train[0:12500] = 1

# do the same for test set
x_test = []
with io.open('../preprocessed_data/imdb_test.txt','r',encoding='utf-8') as f:
    lines = f.readlines()
for line in lines:
    line = line.strip()
    line = line.split(' ')
    line = np.asarray(line,dtype=np.int)

    line[line>vocab_size] = 0

    x_test.append(line)
y_test = np.zeros((25000,))
y_test[0:12500] = 1


# no_of_hidden_units = 500
vocab_size += 1

rnn = RNN_model(vocab_size, no_of_hidden_units)
# load language model
if is_cuda:
    language_model = torch.load('../3a/language_V2.model')
else:
    language_model = torch.load('../3a/language_V2.model', map_location='cpu')



optimizer: adam | LR: 0.001000 | batch_size: 200 | vocab_size: 8000 | no_of_hidden_units: 500 | no_of_epochs: 21 | sequence_length_train: 50
RNN 3C
Cuda is False
Cuda is False


In [6]:
rnn.state_dict()

OrderedDict([('embedding.weight',
              tensor([[-0.1073, -0.1155,  0.5082,  ...,  0.8217, -1.0104, -0.1178],
                      [ 2.7164, -0.1381,  0.3082,  ...,  1.1815,  0.4945, -1.2260],
                      [ 1.9945, -1.0688, -1.4107,  ...,  0.8256, -0.1865,  0.5362],
                      ...,
                      [-0.3715, -0.4670, -0.4154,  ...,  0.2592, -1.3983, -0.4374],
                      [ 1.3179, -0.4485,  1.6855,  ..., -0.7369,  0.4217, -0.2806],
                      [-1.3731, -2.5174, -0.6080,  ...,  0.8348, -0.7215, -0.9307]])),
             ('lstm1.lstm.weight_ih',
              tensor([[-0.0005, -0.0130,  0.0265,  ...,  0.0251,  0.0054, -0.0160],
                      [ 0.0355,  0.0280,  0.0222,  ...,  0.0138, -0.0323, -0.0172],
                      [ 0.0200,  0.0240,  0.0104,  ..., -0.0227,  0.0105,  0.0047],
                      ...,
                      [ 0.0405,  0.0087,  0.0369,  ...,  0.0350, -0.0063,  0.0095],
                      [ 0.0162,

In [7]:
language_model.state_dict()

OrderedDict([('embedding.weight',
              tensor([[ 1.0143,  0.0549, -0.6880,  ...,  0.1039, -0.7074,  1.7013],
                      [-0.9886,  0.5683,  0.2864,  ...,  0.0773,  0.3926, -0.6369],
                      [-1.1951, -0.2433,  0.5355,  ...,  1.3609, -1.3858, -0.1406],
                      ...,
                      [-1.2130, -2.0552, -0.3732,  ..., -0.5742, -0.8301,  0.6543],
                      [ 1.0060,  1.5565, -0.1840,  ...,  0.2977, -0.9768,  0.6599],
                      [ 0.5798, -1.6114, -0.8774,  ..., -0.5737, -0.2022, -0.0839]])),
             ('lstm1.lstm.weight_ih',
              tensor([[ 0.0674, -0.0651, -0.0603,  ..., -0.0171,  0.1823,  0.0776],
                      [ 0.0200,  0.0122,  0.1033,  ...,  0.0304,  0.0564, -0.0037],
                      [ 0.1556,  0.1745, -0.1754,  ..., -0.0783, -0.0171, -0.1079],
                      ...,
                      [ 0.0591,  0.2026, -0.2111,  ..., -0.1468,  0.0310,  0.0647],
                      [-0.0386,

In [9]:
temp = language_model.embedding.state_dict()

In [10]:
t = dict(temp)['weight']

In [11]:
a = t.numpy()

In [12]:
np.std(a[1])

0.8810352

In [2]:

rnn.embedding.load_state_dict(language_model.embedding.state_dict())
rnn.lstm1.lstm.load_state_dict(language_model.lstm1.lstm.state_dict())
rnn.bn_lstm1.load_state_dict(language_model.bn_lstm1.state_dict())
rnn.lstm2.lstm.load_state_dict(language_model.lstm2.lstm.state_dict())
rnn.bn_lstm2.load_state_dict(language_model.bn_lstm2.state_dict())
rnn.lstm3.lstm.load_state_dict(language_model.lstm3.lstm.state_dict())
rnn.bn_lstm3.load_state_dict(language_model.bn_lstm3.state_dict())

if is_cuda:
    rnn.cuda()
    # rnn = nn.DataParallel(rnn, device_ids=range(torch.cuda.device_count()))
    print(torch.cuda.device_count())
    cudnn.benchmark = True
    # rnn.cuda()


# Just fine tine the last LSTM layers
params = []
# for param in model.embedding.parameters():
#     params.append(param)
# for param in model.lstm1.parameters():
#     params.append(param)
# for param in model.bn_lstm1.parameters():
#     params.append(param)
# for param in model.lstm2.parameters():
#     params.append(param)
# for param in model.bn_lstm2.parameters():
#     params.append(param)
for param in rnn.lstm3.parameters():
    params.append(param)
for param in rnn.bn_lstm3.parameters():
    params.append(param)
for param in rnn.fc_output.parameters():
    params.append(param)


## Define the Model with desired learning rate ##
# opt = 'sgd'
# LR = 0.01
# opt = 'adam'
# LR = 0.001
if(opt=='adam'):
    optimizer = optim.Adam(rnn.parameters(), lr=LR)
elif(opt=='sgd'):
    optimizer = optim.SGD(rnn.parameters(), lr=LR, momentum=0.9)



# batch_size = 200
# no_of_epochs = 20
L_Y_train = len(y_train)
L_Y_test = len(y_test)

train_loss = []
train_accu = []
test_accu = []

In [ ]:
for epoch in range(no_of_epochs):

    # training
    rnn.train()

    epoch_acc = 0.0
    epoch_loss = 0.0

    epoch_counter = 0

    time1 = time.time()
    
    I_permutation = np.random.permutation(L_Y_train)

    for i in range(0, L_Y_train, batch_size):

        x_input2 = [x_train[j] for j in I_permutation[i:i+batch_size]]
        sequence_length = 50
        x_input = np.zeros((batch_size,sequence_length),dtype=np.int)
        for j in range(batch_size):
            x = np.asarray(x_input2[j])
            sl = x.shape[0]
            if(sl < sequence_length):
                x_input[j,0:sl] = x
            else:
                start_index = np.random.randint(sl-sequence_length+1)
                x_input[j,:] = x[start_index:(start_index+sequence_length)]
        y_input = y_train[I_permutation[i:i+batch_size]]

        if is_cuda:
            data = Variable(torch.LongTensor(x_input)).cuda()
            target = Variable(torch.FloatTensor(y_input)).cuda()
        else:
            data = Variable(torch.LongTensor(x_input))
            target = Variable(torch.FloatTensor(y_input))

        optimizer.zero_grad()
        loss, pred = rnn(data,target,train=True)
        loss.backward()

        optimizer.step()   # update weights
        
        prediction = pred >= 0.0
        truth = target >= 0.5
        acc = prediction.eq(truth).sum().cpu().data.numpy()
        epoch_acc += acc
        epoch_loss += loss.data.item()
        epoch_counter += batch_size

    epoch_acc /= epoch_counter
    epoch_loss /= (epoch_counter/batch_size)

    train_loss.append(epoch_loss)
    train_accu.append(epoch_acc)

    print('Train | Epoch:',epoch, "| Epoch_Acc %.2f" % (epoch_acc*100.0), "| Loss %.4f" % epoch_loss, "| Time %.4f" % float(time.time()-time1))

    # ## test
    if((epoch+1)%3)==0:
        print('Testing on Epoch', (epoch+1))
        rnn.eval()

        epoch_acc = 0.0
        epoch_loss = 0.0

        epoch_counter = 0

        time1 = time.time()
        
        I_permutation = np.random.permutation(L_Y_test)

        for i in range(0, L_Y_test, batch_size):
            x_input2 = [x_test[j] for j in I_permutation[i:i+batch_size]]
            sequence_length = 200
            x_input = np.zeros((batch_size,sequence_length),dtype=np.int)
            for j in range(batch_size):
                x = np.asarray(x_input2[j])
                sl = x.shape[0]
                if(sl < sequence_length):
                    x_input[j,0:sl] = x
                else:
                    start_index = np.random.randint(sl-sequence_length+1)
                    x_input[j,:] = x[start_index:(start_index+sequence_length)]
            y_input = y_train[I_permutation[i:i+batch_size]]

            if is_cuda:
                data = Variable(torch.LongTensor(x_input)).cuda()
                target = Variable(torch.FloatTensor(y_input)).cuda()
            else:
                data = Variable(torch.LongTensor(x_input))
                target = Variable(torch.FloatTensor(y_input))

            with torch.no_grad():
                loss, pred = rnn(data,target, train=False)
            
            prediction = pred >= 0.0
            truth = target >= 0.5
            acc = prediction.eq(truth).sum().cpu().data.numpy()
            epoch_acc += acc
            epoch_loss += loss.data.item()
            epoch_counter += batch_size

        epoch_acc /= epoch_counter
        epoch_loss /= (epoch_counter/batch_size)

        test_accu.append(epoch_acc)

        time2 = time.time()
        time_elapsed = time2 - time1

        print("Test:  ", "%.2f" % (epoch_acc*100.0), " | Loss: %.4f" % epoch_loss)

torch.save(rnn,'rnn3C.model')
data = [train_loss,train_accu,test_accu]
data = np.asarray(data)
np.save('data.npy',data)

Train | Epoch: 0 | Epoch_Acc 70.50 | Loss 0.5664 | Time 544.8916


In [15]:
data

tensor([[   7,  855,   48,  ...,    2,   11,   79],
        [   0, 1493,   20,  ...,   22,    3,    1],
        [ 135,   43,    5,  ...,    1,  108,   12],
        ...,
        [  16,    1,  235,  ...,   10, 2682,    6],
        [   2,    5,  179,  ...,  111,  131,    3],
        [  66, 1146,    3,  ...,    1,  126,   15]])

In [16]:
target

tensor([0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,
        0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1.,
        0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
        1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
        1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0.,
        0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0.,
        1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0.,
        0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
        1., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0.,
        1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 0.])